# Getting Play-by-play Data

Most fun uses of this data involve processing play-by-play data in some way, so in this notebook I'll find a way to pull all such data for a season (hopefully).

First, based on the last few notebooks, I think we'll need to get IDs for all the teams.
I *think* we can use team IDs to get game IDs -- which are how we look up a game in the play-by-play data.

In [1]:
from nba_api.stats.static.teams import get_teams
import pandas as pd

In [2]:
teams = get_teams()
print(teams[:5])

[{'id': 1610612737, 'full_name': 'Atlanta Hawks', 'abbreviation': 'ATL', 'nickname': 'Hawks', 'city': 'Atlanta', 'state': 'Atlanta', 'year_founded': 1949}, {'id': 1610612738, 'full_name': 'Boston Celtics', 'abbreviation': 'BOS', 'nickname': 'Celtics', 'city': 'Boston', 'state': 'Massachusetts', 'year_founded': 1946}, {'id': 1610612739, 'full_name': 'Cleveland Cavaliers', 'abbreviation': 'CLE', 'nickname': 'Cavaliers', 'city': 'Cleveland', 'state': 'Ohio', 'year_founded': 1970}, {'id': 1610612740, 'full_name': 'New Orleans Pelicans', 'abbreviation': 'NOP', 'nickname': 'Pelicans', 'city': 'New Orleans', 'state': 'Louisiana', 'year_founded': 2002}, {'id': 1610612741, 'full_name': 'Chicago Bulls', 'abbreviation': 'CHI', 'nickname': 'Bulls', 'city': 'Chicago', 'state': 'Illinois', 'year_founded': 1966}]


In [3]:
team_ids = [team['id'] for team in teams]
print(team_ids)

[1610612737, 1610612738, 1610612739, 1610612740, 1610612741, 1610612742, 1610612743, 1610612744, 1610612745, 1610612746, 1610612747, 1610612748, 1610612749, 1610612750, 1610612751, 1610612752, 1610612753, 1610612754, 1610612755, 1610612756, 1610612757, 1610612758, 1610612759, 1610612760, 1610612761, 1610612762, 1610612763, 1610612764, 1610612765, 1610612766]


Okay, so with team IDs, let's get game IDs.

We'll test with just one team, the Hawks.

In [4]:
from nba_api.stats.endpoints import leaguegamefinder
hawks = [team for team in teams if team['full_name'] == 'Atlanta Hawks'][0]
hawks_games = leaguegamefinder.LeagueGameFinder(team_id_nullable=hawks['id'])
hawks_games

In [5]:
hawks_games = hawks_games.get_data_frames()[0]
hawks_games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22018,1610612737,ATL,Atlanta Hawks,0021800149,2018-11-06,ATL @ CHA,L,239,102,...,0.778,7,30,37,22,13.0,9,22,18,-11.0
1,22018,1610612737,ATL,Atlanta Hawks,0021800128,2018-11-03,ATL vs. MIA,W,239,123,...,0.783,8,29,37,33,11.0,7,16,21,5.0
2,22018,1610612737,ATL,Atlanta Hawks,0021800114,2018-11-01,ATL vs. SAC,L,239,115,...,0.676,10,33,43,26,6.0,6,22,28,-31.0
3,22018,1610612737,ATL,Atlanta Hawks,0021800097,2018-10-30,ATL @ CLE,L,240,114,...,0.647,6,33,39,25,6.0,8,22,29,-22.0
4,22018,1610612737,ATL,Atlanta Hawks,0021800088,2018-10-29,ATL @ PHI,L,240,92,...,0.667,10,37,47,20,10.0,3,13,18,-21.0


Looks pretty good!

The Hawks should have played exactly 82 games in the 2017-08 season (they definitely didn't make the playoffs!).

In [6]:
hawks_games_2017 = hawks_games[hawks_games.SEASON_ID.str[-4:] == '2017']
hawks_games_2017.shape

(92, 28)

Hmm, 10 extra games....
what's the deal?

When were these games played?

In [7]:
(pd.to_datetime(hawks_games_2017.GAME_DATE).min(),
 pd.to_datetime(hawks_games_2017.GAME_DATE).max())

(Timestamp('2017-07-07 00:00:00'), Timestamp('2018-04-10 00:00:00'))

July 7th? Way too early for an NBA game -- the season should start in October.
Are we seeing preseason games?

In [8]:
hawks_games_2017.sort_values('GAME_DATE')[:12]

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
115,22017,1610612737,ATL,Atlanta Hawks,1521700003,2017-07-07,ATL vs. BKN,L,200,72,...,0.692,10,33,43,6,7.0,1,22,28,-3.0
114,22017,1610612737,ATL,Atlanta Hawks,1521700015,2017-07-09,ATL @ NOP,W,201,84,...,0.679,11,23,34,14,10.0,2,14,16,2.0
113,22017,1610612737,ATL,Atlanta Hawks,1521700023,2017-07-10,ATL @ CHI,W,199,75,...,0.765,16,36,52,17,9.0,5,13,22,20.0
112,22017,1610612737,ATL,Atlanta Hawks,1521700037,2017-07-12,ATL vs. NOP,L,200,95,...,0.769,9,20,29,13,9.0,3,12,25,-10.0
111,22017,1610612737,ATL,Atlanta Hawks,1521700053,2017-07-14,ATL vs. HOU,W,199,92,...,0.600,14,27,41,21,10.0,6,14,17,6.0
110,12017,1610612737,ATL,Atlanta Hawks,0011700004,2017-10-01,ATL @ MIA,L,241,90,...,0.815,10,32,42,18,14.0,1,16,25,-6.0
109,12017,1610612737,ATL,Atlanta Hawks,0011700018,2017-10-04,ATL @ CLE,W,242,109,...,0.731,6,39,45,28,6.0,2,20,15,16.0
108,12017,1610612737,ATL,Atlanta Hawks,0011700031,2017-10-06,ATL @ DET,L,240,87,...,0.733,7,30,37,24,2.0,3,14,12,-22.0
107,12017,1610612737,ATL,Atlanta Hawks,0011700050,2017-10-09,ATL vs. MEM,W,241,100,...,0.800,7,39,46,22,8.0,7,26,27,8.4
106,12017,1610612737,ATL,Atlanta Hawks,0011700065,2017-10-12,ATL vs. DAL,L,240,94,...,0.719,10,29,39,21,10.0,2,19,12,-14.0


Just 5 games in July.
What?

Some research and... aha! It's Summer League in Vegas.

The Hawks' real season opener was October 8th against the Mavs -- row 104 above.
But how to differentiate programmatically?
The first digit of season ID looks to be 1 for preseason and 2 for regular season, but it's also 2 for Summer League!
This is maddening.

In [9]:
hawks_games_2017.groupby(hawks_games_2017.SEASON_ID.str[:1].rename('First digit of season ID')).count()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
First digit of season ID,,,,,,,,,,,,,,,,,,,,,
1,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
2,87,87,87,87,87,87,87,87,87,87,...,87,87,87,87,87,87,87,87,87,87


Still, this helps a little.
Can we be confident that this pattern holds?
Let's do the same thing for the Warriors; then we can also see how it looks for the playoffs. 

In [10]:
gsw = [team for team in teams if team['full_name'] == 'Golden State Warriors'][0]
gsw_games = leaguegamefinder.LeagueGameFinder(team_id_nullable=gsw['id'])
gsw_games = gsw_games.get_data_frames()[0]
gsw_games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22018,1610612744,GSW,Golden State Warriors,0021800147,2018-11-05,GSW vs. MEM,W,240,117,...,0.957,8.0,38.0,46.0,29,4.0,4,13.0,25,16.0
1,22018,1610612744,GSW,Golden State Warriors,0021800124,2018-11-02,GSW vs. MIN,W,239,116,...,0.773,12.0,49.0,61.0,31,3.0,9,16.0,16,17.0
2,22018,1610612744,GSW,Golden State Warriors,0021800108,2018-10-31,GSW vs. NOP,W,240,131,...,0.700,12.0,34.0,46.0,39,10.0,4,17.0,24,10.0
3,22018,1610612744,GSW,Golden State Warriors,0021800091,2018-10-29,GSW @ CHI,W,241,149,...,0.731,8.0,41.0,49.0,34,9.0,7,7.0,29,25.0
4,22018,1610612744,GSW,Golden State Warriors,0021800083,2018-10-28,GSW @ BKN,W,239,120,...,0.875,12.0,35.0,47.0,28,11.0,7,12.0,16,6.0


In [11]:
gsw_games_2017 = gsw_games[gsw_games.SEASON_ID.str[-4:] == '2017']
gsw_games_2017.groupby(gsw_games_2017.SEASON_ID.str[:1].rename('First digit of season ID')).count()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
First digit of season ID,,,,,,,,,,,,,,,,,,,,,
1,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
2,88,88,88,88,88,88,88,88,88,88,...,88,88,88,88,88,88,88,88,88,88
4,21,21,21,21,21,21,21,21,21,21,...,21,21,21,21,21,21,21,21,21,21


Oooh exciting.
Hopefully those 4 games (ID 1) are after July but before the season opener (Oct 17) and thus preseason, and the 21 games (ID 4) are playoffs.

Yes, [research](https://www.basketball-reference.com/teams/GSW/2018.html) indeed confirms that the Dubs played 21 playoff games that year.
Now let's check on those first 4 games to see if they're summer league.

In [12]:
gsw_games_2017.sort_values('GAME_DATE').iloc[4:12]

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
132,22017,1610612744,GSW,Golden State Warriors,1521700047,2017-07-13,GSW @ BOS,L,201,69,...,0.719,12.0,22.0,34.0,5,13.0,6,12.0,16,-24.0
131,22017,1610612744,GSW,Golden State Warriors,1521700056,2017-07-14,GSW @ LAC,W,199,109,...,0.615,8.0,35.0,43.0,15,8.0,5,13.0,17,9.0
130,12017,1610612744,GSW,Golden State Warriors,0011700001,2017-09-30,GSW vs. DEN,L,240,102,...,0.833,17.0,36.0,53.0,17,13.0,4,14.0,28,-6.0
129,12017,1610612744,GSW,Golden State Warriors,0011700024,2017-10-05,GSW vs. MIN,L,240,95,...,0.520,8.0,36.0,44.0,27,3.0,8,19.0,21,-7.6
128,12017,1610612744,GSW,Golden State Warriors,0011700044,2017-10-08,GSW @ MIN,W,240,142,...,0.800,7.0,26.0,33.0,32,16.0,10,22.0,31,32.0
127,12017,1610612744,GSW,Golden State Warriors,0011700077,2017-10-13,GSW vs. SAC,W,241,117,...,0.588,21.0,26.0,47.0,34,4.0,4,12.0,15,11.0
126,22017,1610612744,GSW,Golden State Warriors,0021700002,2017-10-17,GSW vs. HOU,L,241,121,...,0.905,6.0,35.0,41.0,34,5.0,9,17.0,25,-1.0
125,22017,1610612744,GSW,Golden State Warriors,0021700025,2017-10-20,GSW @ NOP,W,241,128,...,0.800,10.0,39.0,49.0,29,6.0,8,18.0,23,8.0


Yep!
So I feel pretty good saying that the first digit denotes:
    - 1 -> preseason
    - 2 -> regular season or summer league
    - 4 -> playoffs

But still, how to separate regular season from Summer League?
The hacky way would be by date (Summer League is probably always over by October, when the regular season starts).
But so hacky.

Above, it looks like the first two digits of game ID are different in Summer League.
Let's look into it more.

In [13]:
gsw_games_2017.sort_values('GAME_DATE').head(8)

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
136,22017,1610612744,GSW,Golden State Warriors,1521700014,2017-07-08,GSW vs. PHI,L,200,93,...,0.667,11.0,24.0,35.0,22,9.0,6,14.0,25,-2.0
135,22017,1610612744,GSW,Golden State Warriors,1521700028,2017-07-10,GSW @ CLE,L,202,74,...,0.600,11.0,32.0,43.0,13,11.0,4,13.0,30,-17.0
134,22017,1610612744,GSW,Golden State Warriors,1521700036,2017-07-11,GSW @ MIN,L,220,76,...,0.864,11.0,31.0,42.0,16,12.0,12,18.0,24,0.0
133,22017,1610612744,GSW,Golden State Warriors,1521700040,2017-07-12,GSW @ MIN,W,200,77,...,0.630,6.0,42.0,48.0,15,3.0,6,19.0,20,8.0
132,22017,1610612744,GSW,Golden State Warriors,1521700047,2017-07-13,GSW @ BOS,L,201,69,...,0.719,12.0,22.0,34.0,5,13.0,6,12.0,16,-24.0
131,22017,1610612744,GSW,Golden State Warriors,1521700056,2017-07-14,GSW @ LAC,W,199,109,...,0.615,8.0,35.0,43.0,15,8.0,5,13.0,17,9.0
130,12017,1610612744,GSW,Golden State Warriors,0011700001,2017-09-30,GSW vs. DEN,L,240,102,...,0.833,17.0,36.0,53.0,17,13.0,4,14.0,28,-6.0
129,12017,1610612744,GSW,Golden State Warriors,0011700024,2017-10-05,GSW vs. MIN,L,240,95,...,0.520,8.0,36.0,44.0,27,3.0,8,19.0,21,-7.6


It's `15` for all six Summer League games.

In [14]:
gsw_games_2017.groupby(gsw_games_2017.GAME_ID.str[:2].rename('GAME_ID_PREFIX')).count()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
GAME_ID_PREFIX,,,,,,,,,,,,,,,,,,,,,
00,107,107,107,107,107,107,107,107,107,107,...,107,107,107,107,107,107,107,107,107,107
15,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6


Yep, it's 00 everywhere else!

According to this theory:

| GAME_ID_PREFIX | SEASON_ID_PREFIX | Meaning |
| -- | -- | -- |
| 00 | 1 | Preseason |
| 00 | 2 | Regular Season |
| 15 | 2 | Summer League |
| 00 | 4 | Playoffs |

<br>
Now let's test the whole thing on the Warriors and the Hawks.

In [15]:
gsw_games_2017.groupby([gsw_games_2017.GAME_ID.str[:2].rename('GAME_ID_PREFIX'),
                        gsw_games_2017.SEASON_ID.str[:1].rename('SEASON_ID_PREFIX')]).count()

SEASON_ID  TEAM_ID  TEAM_ABBREVIATION  \
GAME_ID_PREFIX SEASON_ID_PREFIX                                          
00             1                         4        4                  4   
               2                        82       82                 82   
               4                        21       21                 21   
15             2                         6        6                  6   

                                 TEAM_NAME  GAME_ID  GAME_DATE  MATCHUP  WL  \
GAME_ID_PREFIX SEASON_ID_PREFIX                                               
00             1                         4        4          4        4   4   
               2                        82       82         82       82  82   
               4                        21       21         21       21  21   
15             2                         6        6          6        6   6   

                                 MIN  PTS     ...      FT_PCT  OREB  DREB  \
GAME_ID_PREFIX SEASON_ID_PREFIX               ...                           
00             1                   4    4     ...           4     4     4   
               2                  82   82     ...          82    82    82   
               4                  21   21     ...          21    21    21   
15             2                   6    6     ...           6     6     6   

                                 REB  AST  STL  BLK  TOV  PF  PLUS_MINUS  
GAME_ID_PREFIX SEASON_ID_PREFIX                                           
00             1                   4    4    4    4    4   4           4  
               2                  82   82   82   82   82  82          82  
               4                  21   21   21   21   21  21          21  
15             2                   6    6    6    6    6   6           6  

[4 rows x 28 columns]

In [16]:
hawks_games_2017.groupby([hawks_games_2017.GAME_ID.str[:2].rename('GAME_ID_PREFIX'),
                          hawks_games_2017.SEASON_ID.str[:1].rename('SEASON_ID_PREFIX')]).count()

SEASON_ID  TEAM_ID  TEAM_ABBREVIATION  \
GAME_ID_PREFIX SEASON_ID_PREFIX                                          
00             1                         5        5                  5   
               2                        82       82                 82   
15             2                         5        5                  5   

                                 TEAM_NAME  GAME_ID  GAME_DATE  MATCHUP  WL  \
GAME_ID_PREFIX SEASON_ID_PREFIX                                               
00             1                         5        5          5        5   5   
               2                        82       82         82       82  82   
15             2                         5        5          5        5   5   

                                 MIN  PTS     ...      FT_PCT  OREB  DREB  \
GAME_ID_PREFIX SEASON_ID_PREFIX               ...                           
00             1                   5    5     ...           5     5     5   
               2                  82   82     ...          82    82    82   
15             2                   5    5     ...           5     5     5   

                                 REB  AST  STL  BLK  TOV  PF  PLUS_MINUS  
GAME_ID_PREFIX SEASON_ID_PREFIX                                           
00             1                   5    5    5    5    5   5           5  
               2                  82   82   82   82   82  82          82  
15             2                   5    5    5    5    5   5           5  

[3 rows x 28 columns]

I'm feeling pretty good about this!
Let's see if every team has 82 regular season games using this method.

In [17]:
rows = []
for team in teams:
    team_id = team['id']
    team_games = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id)
    team_games = team_games.get_data_frames()[0]
    team_games_2017 = team_games[team_games.SEASON_ID.str[-4:] == '2017']
    reg_season_mask = ((team_games_2017.SEASON_ID.str[:1] == '2') &
                       (team_games_2017.GAME_ID.str[:2] == '00'))
    reg_season_2017_games = team_games_2017[reg_season_mask]
    rows.append((team['full_name'], reg_season_2017_games.shape[0]))
    
pretty_df = pd.DataFrame(rows, columns=('Team', 'Number of Regular Season Games'))
pretty_df

,Team,Number of Regular Season Games
0,Atlanta Hawks,82
1,Boston Celtics,82
2,Cleveland Cavaliers,82
3,New Orleans Pelicans,82
4,Chicago Bulls,82
5,Dallas Mavericks,82
6,Denver Nuggets,82
7,Golden State Warriors,82
8,Houston Rockets,82
9,Los Angeles Clippers,82


**SUCCESS**

Let's see if it works for playoff games as well.

In [19]:
rows = []
for team in teams:
    team_id = team['id']
    team_games = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id)
    team_games = team_games.get_data_frames()[0]
    team_games_2017 = team_games[team_games.SEASON_ID.str[-4:] == '2017']
    reg_season_mask = ((team_games_2017.SEASON_ID.str[:1] == '4') &
                       (team_games_2017.GAME_ID.str[:2] == '00'))
    reg_season_2017_games = team_games_2017[reg_season_mask]
    rows.append((team['full_name'], reg_season_2017_games.shape[0]))
    
pretty_df = pd.DataFrame(rows, columns=('Team', 'Number of Playoff Games'))
pretty_df

,Team,Number of Playoff Games
0,Atlanta Hawks,0
1,Boston Celtics,19
2,Cleveland Cavaliers,22
3,New Orleans Pelicans,9
4,Chicago Bulls,0
5,Dallas Mavericks,0
6,Denver Nuggets,0
7,Golden State Warriors,21
8,Houston Rockets,17
9,Los Angeles Clippers,0


Looks right to me!

So now let's get full regular season- and playoff-datasets for 2017-18.

Would this work?

In [20]:
all_da_games = leaguegamefinder.LeagueGameFinder().get_data_frames()[0]

Add a column for playoffs, summer league, preseason, regular season, or other.

In [21]:
def make_subseason(row):
    game_id_prefix = row['GAME_ID'][:2]
    season_id_prefix = row['SEASON_ID'][:1]
    if game_id_prefix == '00' and season_id_prefix == '1':
        return 'preseason'
    elif game_id_prefix == '00' and season_id_prefix == '2':
        return 'regular_season'
    elif game_id_prefix == '00' and season_id_prefix == '4':
        return 'playoffs'
    elif game_id_prefix == '15' and season_id_prefix == '2':
        return 'summer league'
    else:
        return 'other'
all_da_games['subseason'] = all_da_games.apply(make_subseason, axis=1)

Make sure it looks about right.

In [22]:
all_da_games.groupby([all_da_games.SEASON_ID.str[-4:].rename('SEASON'),
                     all_da_games.subseason]).GAME_ID.count().to_frame()

GAME_ID
SEASON subseason              
2011   other               311
       playoffs            168
       regular_season     1030
2012   other              1389
       playoffs            170
       preseason           232
       regular_season     2458
       summer league       120
2013   other              1436
       playoffs            178
       preseason           230
       regular_season     2460
       summer league       122
2014   other              1502
       playoffs            162
       preseason           236
       regular_season     2460
       summer league       134
2015   other              1550
       playoffs            172
       preseason           218
       regular_season     2460
       summer league       134
2016   other              1711
       playoffs            158
       preseason           204
       regular_season     2460
       summer league       134
2017   other              1864
       playoffs            164
       preseason           157
       regular_season     2460
       summer league       134
2018   other               596
       preseason           158
       regular_season      304
       summer league       164

This looks pretty good to me.

I'm not sure what 'other' is, but my guess is international and WNBA; I ran into some of those games when exploring earlier.

The only odd thing is that there are 2460 regular season games per season -- I would expect there to be 1230 (82 * 30 / 2).
It must be the case that each game is listed twice, possibly with home/away teams reveresed.
Let's look.

In [23]:
all_da_games.sort_values('GAME_ID', ascending=False).head(2)

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,subseason
1418,42017,1612709920,RAP,Raptors 905,2041700402,2018-04-10,RAP vs. AUS,L,241,76,...,11,36,47,17,10,7,21,23,-22.0,other
1417,42017,1612709890,AUS,Austin Spurs,2041700402,2018-04-10,AUS @ RAP,W,239,98,...,10,38,48,28,14,8,16,22,22.0,other


Yep, just looking at the top two "games", they're actually just the same game with home/away flipped.
They even have the same ID.

Let's see if every game ID occurs twice and exactly twice.

In [24]:
game_ids = all_da_games.groupby('GAME_ID', as_index=False).SEASON_ID.count().rename(columns={'SEASON_ID':'OCCURRENCES'})
game_ids.groupby('OCCURRENCES').count()

,GAME_ID
OCCURRENCES,
1,5
2,14985
3,3
4,4


Well, there's some weird ones, but 12 out of 14997 (.08%) is not too bad.

Because I'm always paranoid about external data sources disappearing, and this data isn't that big, I'm just going to save this.

In [26]:
all_da_games.to_csv('data/play_by_play.csv', index=False)